# Projeto SQL Corporativo
Este notebook demonstra o uso de SQL puro em um ambiente Python (DuckDB), com consultas analíticas e relacionamentos entre tabelas de uma empresa fictícia.

**Configuração**

In [11]:
!pip install jupysql duckdb-engine
%load_ext sql

%sql duckdb://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Connecting and switching to connection 'duckdb:///duckdb_database.duckdb'

**CRIAR TABELA**

Cria uma tabela chamada funcionarios para armazenar informações sobre os colaboradores de uma empresa.
Antes de criá-la, o comando DROP TABLE IF EXISTS remove a tabela caso ela já exista, evitando erros de duplicação.

A estrutura da tabela define os seguintes campos:

| Coluna          | Tipo de dado           | Descrição                                             |
| --------------- | ---------------------- | ----------------------------------------------------- |
| `id`            | `INT` (chave primária) | Identificador único de cada funcionário.              |
| `nome`          | `VARCHAR(100)`         | Nome completo do funcionário.                         |
| `cargo`         | `VARCHAR(50)`          | Função ou cargo ocupado na empresa.                   |
| `departamento`  | `VARCHAR(50)`          | Setor ou departamento ao qual o funcionário pertence. |
| `salario`       | `DECIMAL(10,2)`        | Salário do funcionário, com duas casas decimais.      |
| `cidade`        | `VARCHAR(50)`          | Cidade onde o funcionário trabalha.                   |
| `idade`         | `INT`                  | Idade atual do funcionário.                           |
| `data_admissao` | `DATE`                 | Data em que o funcionário foi contratado.             |

In [12]:
%%sql

CREATE TABLE departamentos (
    id INT PRIMARY KEY,
    nome VARCHAR(50)
);

CREATE TABLE cidades (
    id INT PRIMARY KEY,
    nome VARCHAR(50)
);

CREATE TABLE cargos (
    id INT PRIMARY KEY,
    nome VARCHAR(50),
    departamento_id INT,
    FOREIGN KEY (departamento_id) REFERENCES departamentos(id)
);

-- Tabela principal
CREATE TABLE funcionarios (
    id INT PRIMARY KEY,
    nome VARCHAR(100),
    cargo_id INT,
    cidade_id INT,
    salario DECIMAL(10,2),
    idade INT,
    data_admissao DATE,
    FOREIGN KEY (cargo_id) REFERENCES cargos(id),
    FOREIGN KEY (cidade_id) REFERENCES cidades(id)
);

Running query in 'duckdb:///duckdb_database.duckdb'

Count


**INSERIR DADOS**

O comando INSERT INTO funcionarios VALUES (...) adiciona 10 registros à tabela funcionarios, preenchendo as colunas definidas anteriormente (id, nome, cargo, departamento, salário, cidade, idade e data de admissão).

Cada linha representa um funcionário, com suas respectivas informações pessoais e profissionais.

| id | nome           | cargo               | departamento | salário | cidade         | idade | data_admissao |
| -- | -------------- | ------------------- | ------------ | ------- | -------------- | ----- | ------------- |
| 1  | Ana Souza      | Analista de Dados   | TI           | 6500    | São Paulo      | 29    | 2020-05-10    |
| 2  | João Lima      | Desenvolvedor       | TI           | 7200    | Rio de Janeiro | 33    | 2018-03-14    |
| 3  | Maria Santos   | Designer            | Marketing    | 4800    | Curitiba       | 26    | 2021-09-01    |
| 4  | Pedro Oliveira | Gerente de Projetos | TI           | 9500    | São Paulo      | 41    | 2015-01-22    |
| 5  | Júlia Martins  | Analista Financeira | Financeiro   | 5800    | Belo Horizonte | 30    | 2019-06-18    |
| 6  | Lucas Almeida  | Desenvolvedor       | TI           | 7100    | Porto Alegre   | 28    | 2020-11-05    |
| 7  | Carla Dias     | Coordenadora MKT    | Marketing    | 8500    | São Paulo      | 35    | 2016-02-27    |
| 8  | Rafael Gomes   | Analista de Dados   | TI           | 6700    | Curitiba       | 27    | 2022-01-15    |
| 9  | Fernanda Costa | Estagiária          | Financeiro   | 2200    | Recife         | 22    | 2023-03-10    |
| 10 | Bruno Ferreira | Desenvolvedor       | TI           | 7500    | Rio de Janeiro | 31    | 2017-04-30    |

In [13]:
%%sql

-- Departamentos
INSERT INTO departamentos (id, nome) VALUES (1, 'TI');
INSERT INTO departamentos (id, nome) VALUES (2, 'Marketing');
INSERT INTO departamentos (id, nome) VALUES (3, 'Financeiro');

-- Cidades
INSERT INTO cidades (id, nome) VALUES (1, 'São Paulo');
INSERT INTO cidades (id, nome) VALUES (2, 'Rio de Janeiro');
INSERT INTO cidades (id, nome) VALUES (3, 'Curitiba');
INSERT INTO cidades (id, nome) VALUES (4, 'Belo Horizonte');
INSERT INTO cidades (id, nome) VALUES (5, 'Porto Alegre');
INSERT INTO cidades (id, nome) VALUES (6, 'Recife');

-- Cargos
INSERT INTO cargos (id, nome, departamento_id) VALUES (1, 'Analista de Dados', 1);
INSERT INTO cargos (id, nome, departamento_id) VALUES (2, 'Desenvolvedor', 1);
INSERT INTO cargos (id, nome, departamento_id) VALUES (3, 'Gerente de Projetos', 1);
INSERT INTO cargos (id, nome, departamento_id) VALUES (4, 'Designer', 2);
INSERT INTO cargos (id, nome, departamento_id) VALUES (5, 'Coordenadora MKT', 2);
INSERT INTO cargos (id, nome, departamento_id) VALUES (6, 'Analista Financeira', 3);
INSERT INTO cargos (id, nome, departamento_id) VALUES (7, 'Estagiária', 3);

-- Funcionários
INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES (1, 'Ana Souza', 1, 1, 6500, 29, '2020-05-10');
INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES (2, 'João Lima', 2, 2, 7200, 33, '2018-03-14');
INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES (3, 'Maria Santos', 4, 3, 4800, 26, '2021-09-01');
INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES (4, 'Pedro Oliveira', 3, 1, 9500, 41, '2015-01-22');
INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES (5, 'Júlia Martins', 6, 4, 5800, 30, '2019-06-18');
INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES (6, 'Lucas Almeida', 2, 5, 7100, 28, '2020-11-05');
INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES (7, 'Carla Dias', 5, 1, 8500, 35, '2016-02-27');
INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES (8, 'Rafael Gomes', 1, 3, 6700, 27, '2022-01-15');
INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES (9, 'Fernanda Costa', 7, 6, 2200, 22, '2023-03-10');
INSERT INTO funcionarios (id, nome, cargo_id, cidade_id, salario, idade, data_admissao) VALUES (10, 'Bruno Ferreira', 2, 2, 7500, 31, '2017-04-30');

Running query in 'duckdb:///duckdb_database.duckdb'

Count


**Mostre todos os registros da tabela**

In [17]:
%%sql
SELECT * FROM funcionarios;

Running query in 'duckdb:///duckdb_database.duckdb'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
1,Ana Souza,1,1,6500.00,29,2020-05-10
2,João Lima,2,2,7200.00,33,2018-03-14
3,Maria Santos,4,3,4800.00,26,2021-09-01
4,Pedro Oliveira,3,1,9500.00,41,2015-01-22
5,Júlia Martins,6,4,5800.00,30,2019-06-18
6,Lucas Almeida,2,5,7100.00,28,2020-11-05
7,Carla Dias,5,1,8500.00,35,2016-02-27
8,Rafael Gomes,1,3,6700.00,27,2022-01-15
9,Fernanda Costa,7,6,2200.00,22,2023-03-10
10,Bruno Ferreira,2,2,7500.00,31,2017-04-30


Liste os nomes e cargos de quem trabalha no departamento TI.

In [19]:
%%sql
SELECT
    f.nome AS funcionario,
    c.nome AS cargo,
    d.nome AS departamento
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
JOIN departamentos d ON c.departamento_id = d.id
WHERE d.nome = 'TI';

Running query in 'duckdb:///duckdb_database.duckdb'

funcionario,cargo,departamento
Ana Souza,Analista de Dados,TI
João Lima,Desenvolvedor,TI
Pedro Oliveira,Gerente de Projetos,TI
Lucas Almeida,Desenvolvedor,TI
Rafael Gomes,Analista de Dados,TI
Bruno Ferreira,Desenvolvedor,TI


Mostre apenas os nomes distintos das cidades cadastradas.


In [21]:
%%sql
SELECt DISTINCT nome
FROM cidades;

Running query in 'duckdb:///duckdb_database.duckdb'

nome
São Paulo
Rio de Janeiro
Curitiba
Belo Horizonte
Recife
Porto Alegre


Exiba os nomes e salários dos funcionários com salário acima de 7000.


In [22]:
%%sql
SELECT nome, salario
FROM funcionarios
WHERE salario > 7000;

Running query in 'duckdb:///duckdb_database.duckdb'

nome,salario
João Lima,7200.00
Pedro Oliveira,9500.00
Lucas Almeida,7100.00
Carla Dias,8500.00
Bruno Ferreira,7500.00


Mostre os funcionários com idade entre 25 e 35 anos.


In [23]:
%%sql
SELECT *
FROM funcionarios
WHERE idade BETWEEN 25 AND 35;

Running query in 'duckdb:///duckdb_database.duckdb'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
1,Ana Souza,1,1,6500.00,29,2020-05-10
2,João Lima,2,2,7200.00,33,2018-03-14
3,Maria Santos,4,3,4800.00,26,2021-09-01
5,Júlia Martins,6,4,5800.00,30,2019-06-18
6,Lucas Almeida,2,5,7100.00,28,2020-11-05
7,Carla Dias,5,1,8500.00,35,2016-02-27
8,Rafael Gomes,1,3,6700.00,27,2022-01-15
10,Bruno Ferreira,2,2,7500.00,31,2017-04-30


Liste os funcionários que trabalham em “São Paulo” ou “Curitiba”.


In [24]:
%%sql
SELECT
    f.id,
    f.nome,
    c.nome AS cidade,
    f.salario,
    f.idade,
    f.data_admissao
FROM funcionarios f
JOIN cidades c ON f.cidade_id = c.id
WHERE c.nome IN ('São Paulo', 'Curitiba');

Running query in 'duckdb:///duckdb_database.duckdb'

id,nome,cidade,salario,idade,data_admissao
1,Ana Souza,São Paulo,6500.00,29,2020-05-10
3,Maria Santos,Curitiba,4800.00,26,2021-09-01
4,Pedro Oliveira,São Paulo,9500.00,41,2015-01-22
7,Carla Dias,São Paulo,8500.00,35,2016-02-27
8,Rafael Gomes,Curitiba,6700.00,27,2022-01-15


Exiba os funcionários cujo nome começa com “A”.


In [25]:
%%sql
SELECT *
FROM funcionarios
WHERE nome LIKE 'A%';

Running query in 'duckdb:///duckdb_database.duckdb'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
1,Ana Souza,1,1,6500.00,29,2020-05-10


Liste os funcionários cujo cargo contém a palavra “Analista”.


In [27]:
%%sql
SELECT
    f.id,
    f.nome,
    c.nome AS cargo,
    f.salario,
    f.idade,
    f.data_admissao
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
WHERE c.nome LIKE '%Analista%';

Running query in 'duckdb:///duckdb_database.duckdb'

id,nome,cargo,salario,idade,data_admissao
1,Ana Souza,Analista de Dados,6500.00,29,2020-05-10
5,Júlia Martins,Analista Financeira,5800.00,30,2019-06-18
8,Rafael Gomes,Analista de Dados,6700.00,27,2022-01-15










Mostre os funcionários admitidos depois de 2020.

In [28]:
%%sql
SELECT *
FROM funcionarios
WHERE data_admissao > '2020-12-31';

Running query in 'duckdb:///duckdb_database.duckdb'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
3,Maria Santos,4,3,4800.00,26,2021-09-01
8,Rafael Gomes,1,3,6700.00,27,2022-01-15
9,Fernanda Costa,7,6,2200.00,22,2023-03-10


Atualizando o valor registrado

In [30]:
%%sql

UPDATE funcionarios
SET salario = 7000.00
WHERE cargo_id IN (
    SELECT id
    FROM cargos
    WHERE departamento_id = (
        SELECT id FROM departamentos WHERE nome = 'TI'
    )
);

-- Conferir os resultados
SELECT
    f.nome,
    c.nome AS cargo,
    d.nome AS departamento,
    f.salario
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
JOIN departamentos d ON c.departamento_id = d.id;

Running query in 'duckdb:///duckdb_database.duckdb'

nome,cargo,departamento,salario
Ana Souza,Analista de Dados,TI,7000.00
João Lima,Desenvolvedor,TI,7000.00
Maria Santos,Designer,Marketing,4800.00
Pedro Oliveira,Gerente de Projetos,TI,7000.00
Júlia Martins,Analista Financeira,Financeiro,5800.00
Lucas Almeida,Desenvolvedor,TI,7000.00
Carla Dias,Coordenadora MKT,Marketing,8500.00
Rafael Gomes,Analista de Dados,TI,7000.00
Fernanda Costa,Estagiária,Financeiro,2200.00
Bruno Ferreira,Desenvolvedor,TI,7000.00


Conte quantos funcionários existem na tabela.

In [31]:
%%sql
SELECT COUNT(*) AS total_funcionarios
FROM funcionarios;

Running query in 'duckdb:///duckdb_database.duckdb'

total_funcionarios
10


Calcule o salário médio de todos os funcionários.

In [32]:
%%sql
SELECT AVG(salario) AS salario_medio
FROM funcionarios;

Running query in 'duckdb:///duckdb_database.duckdb'

salario_medio
6330.0


Mostre o maior e o menor salário registrados.

In [33]:
%%sql
SELECT MIN(salario) AS salario_minimo, MAX(salario) AS salario_maximo
FROM funcionarios;

Running query in 'duckdb:///duckdb_database.duckdb'

salario_minimo,salario_maximo
2200.00,8500.00


Exiba a soma total dos salários do departamento de TI.

In [35]:
%%sql
SELECT SUM(f.salario) AS soma_salarios_ti
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
JOIN departamentos d ON c.departamento_id = d.id
WHERE d.nome = 'TI';

Running query in 'duckdb:///duckdb_database.duckdb'

soma_salarios_ti
42000.00


Mostre quantos funcionários há em cada departamento.

In [37]:
%%sql
SELECT d.nome AS departamento, COUNT(*) AS total_funcionarios
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
JOIN departamentos d ON c.departamento_id = d.id
GROUP BY d.nome;

Running query in 'duckdb:///duckdb_database.duckdb'

departamento,total_funcionarios
TI,6
Marketing,2
Financeiro,2


Calcule a média salarial por cidade.

In [38]:
%%sql
SELECT ci.nome AS cidade,
       ROUND(AVG(f.salario), 2) AS media_salarial
FROM funcionarios f
JOIN cidades ci ON f.cidade_id = ci.id
GROUP BY ci.nome
ORDER BY ci.nome;

Running query in 'duckdb:///duckdb_database.duckdb'

cidade,media_salarial
Belo Horizonte,5800.0
Curitiba,5900.0
Porto Alegre,7000.0
Recife,2200.0
Rio de Janeiro,7000.0
São Paulo,7500.0


Exiba apenas os departamentos cuja média salarial é maior que 7000.

In [39]:
%%sql
-- Média salarial por departamento, filtrando médias > 7000 e ordenando
SELECT d.nome AS departamento,
       ROUND(AVG(f.salario), 2) AS media_salarial
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
JOIN departamentos d ON c.departamento_id = d.id
GROUP BY d.nome
HAVING AVG(f.salario) > 7000
ORDER BY media_salarial DESC;

Running query in 'duckdb:///duckdb_database.duckdb'

departamento,media_salarial


Mostre o número de funcionários por cargo, ordenando do maior para o menor.

In [40]:
%%sql
SELECT c.nome AS cargo,
       COUNT(*) AS total_funcionarios
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
GROUP BY c.nome
ORDER BY total_funcionarios DESC;

Running query in 'duckdb:///duckdb_database.duckdb'

cargo,total_funcionarios
Desenvolvedor,3
Analista de Dados,2
Designer,1
Gerente de Projetos,1
Coordenadora MKT,1
Analista Financeira,1
Estagiária,1


Liste os 5 funcionários com os maiores salários.

In [41]:
%%sql
SELECT
    f.nome AS funcionario,
    c.nome AS cargo,
    d.nome AS departamento,
    ci.nome AS cidade,
    f.salario,
    f.idade,
    f.data_admissao
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
JOIN departamentos d ON c.departamento_id = d.id
JOIN cidades ci ON f.cidade_id = ci.id
ORDER BY f.salario DESC
LIMIT 5;

Running query in 'duckdb:///duckdb_database.duckdb'

funcionario,cargo,departamento,cidade,salario,idade,data_admissao
Carla Dias,Coordenadora MKT,Marketing,São Paulo,8500.00,35,2016-02-27
Ana Souza,Analista de Dados,TI,São Paulo,7000.00,29,2020-05-10
Lucas Almeida,Desenvolvedor,TI,Porto Alegre,7000.00,28,2020-11-05
Pedro Oliveira,Gerente de Projetos,TI,São Paulo,7000.00,41,2015-01-22
João Lima,Desenvolvedor,TI,Rio de Janeiro,7000.00,33,2018-03-14


Mostre os 3 funcionários mais antigos (menor data de admissão).

In [42]:
%%sql
SELECT *
FROM funcionarios
ORDER BY data_admissao ASC
LIMIT 3;

Running query in 'duckdb:///duckdb_database.duckdb'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
4,Pedro Oliveira,3,1,7000.00,41,2015-01-22
7,Carla Dias,5,1,8500.00,35,2016-02-27
10,Bruno Ferreira,2,2,7000.00,31,2017-04-30


Exiba os funcionários ordenados por cidade (ascendente) e salário (descendente).

In [44]:
%%sql
SELECT
    f.nome AS funcionario,
    c.nome AS cargo,
    d.nome AS departamento,
    ci.nome AS cidade,
    f.salario,
    f.idade,
    f.data_admissao
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
JOIN departamentos d ON c.departamento_id = d.id
JOIN cidades ci ON f.cidade_id = ci.id
ORDER BY ci.nome ASC, f.salario DESC;

Running query in 'duckdb:///duckdb_database.duckdb'

funcionario,cargo,departamento,cidade,salario,idade,data_admissao
Júlia Martins,Analista Financeira,Financeiro,Belo Horizonte,5800.00,30,2019-06-18
Rafael Gomes,Analista de Dados,TI,Curitiba,7000.00,27,2022-01-15
Maria Santos,Designer,Marketing,Curitiba,4800.00,26,2021-09-01
Lucas Almeida,Desenvolvedor,TI,Porto Alegre,7000.00,28,2020-11-05
Fernanda Costa,Estagiária,Financeiro,Recife,2200.00,22,2023-03-10
João Lima,Desenvolvedor,TI,Rio de Janeiro,7000.00,33,2018-03-14
Bruno Ferreira,Desenvolvedor,TI,Rio de Janeiro,7000.00,31,2017-04-30
Carla Dias,Coordenadora MKT,Marketing,São Paulo,8500.00,35,2016-02-27
Ana Souza,Analista de Dados,TI,São Paulo,7000.00,29,2020-05-10
Pedro Oliveira,Gerente de Projetos,TI,São Paulo,7000.00,41,2015-01-22


Mostre os funcionários do departamento de TI com salário acima da média do departamento.

In [50]:
%%sql
WITH media_departamento AS (
    SELECT c.departamento_id,
           AVG(f.salario) AS media_salarial
    FROM funcionarios f
    JOIN cargos c ON f.cargo_id = c.id
    GROUP BY c.departamento_id
)
SELECT
    f.nome AS funcionario,
    c.nome AS cargo,
    d.nome AS departamento,
    f.salario
FROM funcionarios f
JOIN cargos c ON f.cargo_id = c.id
JOIN departamentos d ON c.departamento_id = d.id
JOIN media_departamento md ON d.id = md.departamento_id
WHERE f.salario > md.media_salarial
ORDER BY d.nome, f.salario DESC;

Running query in 'duckdb:///duckdb_database.duckdb'

funcionario,cargo,departamento,salario
Júlia Martins,Analista Financeira,Financeiro,5800.00
Carla Dias,Coordenadora MKT,Marketing,8500.00


Calcule a diferença de salário entre o maior e o menor valor da tabela.

In [54]:
%%sql
SELECT MAX(salario) - MIN(salario) AS diferenca_salarial
FROM funcionarios;

Running query in 'duckdb:///duckdb_database.duckdb'

diferenca_salarial
6300.00


Mostre os funcionários que ganham acima da média geral.

In [55]:
%%sql
SELECT funcionarios.*
FROM funcionarios
WHERE salario > (SELECT AVG(salario) FROM funcionarios);

Running query in 'duckdb:///duckdb_database.duckdb'

id,nome,cargo_id,cidade_id,salario,idade,data_admissao
1,Ana Souza,1,1,7000.00,29,2020-05-10
2,João Lima,2,2,7000.00,33,2018-03-14
4,Pedro Oliveira,3,1,7000.00,41,2015-01-22
6,Lucas Almeida,2,5,7000.00,28,2020-11-05
7,Carla Dias,5,1,8500.00,35,2016-02-27
8,Rafael Gomes,1,3,7000.00,27,2022-01-15
10,Bruno Ferreira,2,2,7000.00,31,2017-04-30
